### 1. Data Wrangling of Audible Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pd.options.display.max_rows = 50000

In [ ]:
#import ydata_profiling as yp

In [2]:
#read data
audible_data = pd.read_csv("data/audible_dataset/audible_uncleaned.csv")

Below, I've tried out **`ydata-profiling`** pacakge to generate a basic overview report on the datafile. It highlights a lot of features in the dataset like number of duplicates, missing values, encoding, language consistency etc.  
[This](https://www.blog.datahut.co/post/data-cleaning-techniques) is a great blog on cleaning scraped data.

In [ ]:
#original_report = yp.ProfileReport(audible_data, title = "Prelim Analysis")
#original_report.to_file("audible_prelim_report.html")

In [3]:
audible_data.sample(4)

,name,author,narrator,time,releasedate,language,stars,price
47882,The Tiger Came to the Mountains,Writtenby:SilviaMoreno-Garcia,Narratedby:CynthiaFarrell,50 mins,24-02-22,English,Not rated yet,67.00
72831,Jewels of the Sun,Writtenby:NoraRoberts,Narratedby:PatriciaDaniels,10 hrs and 25 mins,10-06-08,English,5 out of 5 stars1 rating,"1,008.00"
26071,The Misbegotten Son,Writtenby:JackOlsen,Narratedby:KevinPierce,18 hrs and 11 mins,04-12-15,English,Not rated yet,836.00
14293,G/GESCHICHTE - Byzanz - Roms goldene Tochter,Writtenby:GGeschichte,Narratedby:KarstenWolf,2 hrs and 28 mins,23-12-21,german,Not rated yet,233.00


In [4]:
audible_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


In [5]:
audible_data.describe()

,name,author,narrator,time,releasedate,language,stars,price
count,87489,87489,87489,87489,87489,87489,87489,87489
unique,82767,48374,29717,2284,5058,36,665,1011
top,The Art of War,"Writtenby:矢島雅弘,石橋遊",Narratedby:anonymous,2 mins,16-05-18,English,Not rated yet,586.00
freq,20,874,1034,372,773,61884,72417,5533


From the `describe()` output, we can see that there are a total of 87,489 books but only 82,767 of them are unique entries. Some books have multiple rows maybe due to a different language of publication or a different narrator etc.  
Below, I have listed out books with equal to or more than 10 entries in the dataset.

In [6]:
#checking number of occurances of same book
book_counts = audible_data['name'].value_counts()
book_counts[book_counts >= 10]

The Art of War                 20
Sterling Biographies           19
The Odyssey                    16
Sterling Point Books           16
Hamlet                         15
The Prophet                    14
Pride and Prejudice            14
A Christmas Carol              14
The Iliad                      13
As a Man Thinketh              13
The Science of Getting Rich    13
The Picture of Dorian Gray     12
Abraham Lincoln                12
Meditations                    11
The Richest Man in Babylon     11
The Raven                      11
The Prince                     11
Unstoppable                    10
Name: name, dtype: int64

No duplicate rows found.

In [7]:
#check for duplicate rows
audible_data.duplicated().sum()

0

In [8]:
#filter out all book names containing any special characters
booknames_special_chars = audible_data[audible_data.name.str.contains(r'[@#$%+/*]')].drop_duplicates()
#number of books that contain special characters- 592 books
booknames_special_chars.shape

(592, 8)

In [9]:
booknames_special_chars.sample(4)

,name,author,narrator,time,releasedate,language,stars,price
35522,Tinnitus Sound Therapy / Tinnitus Retraining T...,Writtenby:Dr.HannahLiebig,Narratedby:IanBrannan,7 hrs and 4 mins,15-07-21,English,5 out of 5 stars1 rating,669.00
49412,Véra / La Reine Ysabeau,Writtenby:AugustedeVilliersdeL'Isle-Adam,Narratedby:RafaëlReves,37 mins,30-08-10,french,Not rated yet,184.00
58764,The Top 1%,Writtenby:DanStrutzel,Narratedby:DanStrutzel,5 hrs and 10 mins,28-03-17,English,Not rated yet,656.00
902,Mercy Watson #3,Writtenby:KateDiCamillo,Narratedby:RonMcLarty,20 mins,08-06-07,English,5 out of 5 stars2 ratings,200.00


In [10]:
#Author column
#remove the phrase "Writtenby:"
audible_data['author'] = audible_data['author'].str.replace(pat = "Writtenby:", repl = "")

In [11]:
audible_data.author

0        GeronimoStilton
1            RickRiordan
2             JeffKinney
3            RickRiordan
4            RickRiordan
              ...       
87484       ChrisStewart
87485      StephenO'Shea
87486          MarkTwain
87487     LaurenceSterne
87488      MarkKurlansky
Name: author, Length: 87489, dtype: object

Stack overflow thread to understand the Regex: [SO](https://stackoverflow.com/questions/199059/a-pythonic-way-to-insert-a-space-before-capital-letters)

The line below adds space between the first, middle (if any) and last name of all authors.  
Ofc, this works only for names mentioned in English. We also have names in different languages.

In [12]:
#Add space between the first, middle and last names of Authors.
#e.g. JaneAustin becomes Jane Austin
audible_data['author'] = audible_data['author'].str.replace(pat = r"(\w)([A-Z])", repl = r"\1 \2", regex = True)

Some books have multiple authors. So, below the 'author' column has been split into multiple columns with author1, author2, author3..etc. each column with a single name.  
Separator "," has been used to split the column.

In [13]:
audible_data2 = pd.concat( [audible_data['name'], 
                            audible_data['author'].str.split(',', expand = True).add_prefix('author'),
                            audible_data.loc[:,['narrator', 'time', 'releasedate', 'language', 'stars', 'price']]], 
                            axis = 1)

In [14]:
audible_data2.sample(4)

,name,author0,author1,author2,author3,narrator,time,releasedate,language,stars,price
5899,"Good Morning, Gorillas",Mary Pope Osborne,None,None,None,Narratedby:MaryPopeOsborne,40 mins,09-06-07,English,Not rated yet,301.00
84250,Dangerous Destiny,Suzanne Brockmann,Melanie Brockmann,None,None,Narratedby:MelanieBrockmann,1 hr and 15 mins,07-10-14,English,Not rated yet,425.00
36273,The Neurotic Character,Alfred Adler,None,None,None,Narratedby:LeightonPugh,14 hrs and 13 mins,13-02-22,English,Not rated yet,"1,074.00"
60704,Die Insel,Andreas Zwengel,None,None,None,Narratedby:PeterBocek,4 hrs and 32 mins,04-01-22,german,Not rated yet,334.00


In the following code, I just wanted to check how many of the new author name columns- author1, author2, author3 contain null values.  
If most of the rows are vacant, then there's no point creating additional columns for the same.  
I've kept this part in the notebook to show how split can be done if need arises!  


In [15]:
#number of missing values in author columns
audible_data2.loc[:,['author0','author1','author2','author3']].isnull().sum()

author0        0
author1    73762
author2    85135
author3    86713
dtype: int64

In [16]:
#remove "Narratedby:" from Narrator column
audible_data2['narrator'] = audible_data2['narrator'].str.replace(pat = "Narratedby:", repl = "")

In [17]:
#add space between first and last name of the Narrator
#e.g. JaneAustin becomes Jane Austin
audible_data2['narrator'] = audible_data2['narrator'].str.replace(pat = r"(\w)([A-Z])", repl = r"\1 \2", regex = True)

In [ ]:
audible_data2.sample(5)

## Time column

References:  
1. [SO: Convert string to HH:MM](https://stackoverflow.com/questions/47097447/convert-string-to-hhmm-time-in-python)  
2. [SO: Separate time from date component](https://stackoverflow.com/questions/32375471/pandas-convert-strings-to-time-without-date)

In [18]:
#make a copy of time column to understand all kind of formats in which data is present
time_column = audible_data2['time']
time_column_copy = time_column.copy()

Since the 'time' column is of char type, first we need to see all distinct formats in which time has been mentioned.  
For example, some rows might have a format- '7 hrs 22 mins' while others with '7 hr 22 mins' (note the missing 's' in hr), '9 hrs' and so on..  
Depending on all formats, we figure out steps to be taken to clean this column.  
The objective is to convert these values to the form **hh:mm**. 

In [ ]:
#replace all numbers with blanks
time_column_copy = time_column_copy.str.replace(pat = r'[0-9]', repl = '', regex = True)
time_column_copy.sample(3)

In [ ]:
#keep only unique patterns
time_column_copy = time_column_copy.drop_duplicates()
time_column_copy

'Time' column contains following formats:  
1. 7 hrs and 54 mins
2. 9 hrs
3. 7 mins

### A series with some time values

In [21]:
bus_time = pd.Series(['7 hrs and 20 mins', '9 hrs', '12 hr and 30 min', '4 min', 'less than 1 min', 'less than 14 mins'])

In [70]:
time_copy = bus_time.copy()

### Rows containing 'and'

In [71]:
time_copy

0    7 hrs and 20 mins
1                9 hrs
2     12 hr and 30 min
3                4 min
4      less than 1 min
5    less than 14 mins
dtype: object

In [72]:
#find the indices of rows containing format 'hrs and mins'
and_row_index = time_copy[time_copy.str.contains('and')].index
#replace all words with ''
time_copy[and_row_index] = time_copy[and_row_index].str.replace(pat = ' and | hrs| hr| mins| min', repl = '', regex = True)
#left-pad the remaining digits with 0 to get the pattern like 00:07 [hh:mm]
time_copy[and_row_index] = time_copy[and_row_index].str.zfill(4)
#add a colon between first 2 and last 2 digits to make the pattern hh:mm
time_copy[and_row_index] = time_copy[and_row_index].str[:2] + ':' + time_copy[and_row_index].str[2:]

In [73]:
time_copy

0                07:20
1                9 hrs
2                12:30
3                4 min
4      less than 1 min
5    less than 14 mins
dtype: object

In [46]:
#making a copy for minute rows manipulation
time_copy2 = time_copy.copy()

In [47]:
time_copy2

0                07:20
1                9 hrs
2                12:30
3                4 min
4      less than 1 min
5    less than 14 mins
dtype: object

### Rows containing 'less than'

In [74]:
#rows with 'less than x minute'
lt_row_index = time_copy[time_copy.str.contains('less than')].index
time_copy[lt_row_index] = time_copy[lt_row_index].str.replace(pat = 'less than ', repl = '', regex = True)

### Now, rows containing 'mins OR min'

In [75]:
min_row_index = time_copy[time_copy.str.contains('mins|min|minute')].index
time_copy[min_row_index] = time_copy[min_row_index].str.replace(pat = ' mins| min', repl = '', regex = True)
time_copy[min_row_index] = time_copy[min_row_index].str.zfill(4)
time_copy[min_row_index] = time_copy[min_row_index].str[:2] + ':' + time_copy[min_row_index].str[2:]
time_copy

0    07:20
1    9 hrs
2    12:30
3    00:04
4    00:01
5    00:14
dtype: object

### rows with 'hrs' or 'hr'

In [84]:
time_copy2 = time_copy.copy()
hrs_row_index = time_copy2[time_copy2.str.contains('hr')].index
hrs_row_index

Int64Index([1], dtype='int64')

In [85]:
#replace hrs or hr with ''
time_copy2[hrs_row_index] = time_copy2[hrs_row_index].str.replace(pat = ' hrs| hr', repl = '', regex = True)
time_copy2

0    07:20
1        9
2    12:30
3    00:04
4    00:01
5    00:14
dtype: object

In [86]:
#check whether the digit is single or double. If between 1-9, then right-pad 2 zeros and left-pad 1 zero. e.g. 09:00
#if double digit, then only right pad 2 zeros. e.g. 16:00
time_copy2[hrs_row_index]

1    9
dtype: object

In [88]:
if time_copy2[hrs_row_index].astype(int) <= 9:
    time_copy2[hrs_row_index] = time_copy2[hrs_row_index].str.zfill(2)
    time_copy2[hrs_row_index] = time_copy2[hrs_row_index].str.ljust(4,'0')
else:
    time_copy2[hrs_row_idex] = time_copy2[hrs_row_index].str.ljust(4, '0')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [82]:
time_copy2[hrs_row_index] = time_copy2[hrs_row_index].str.ljust(4,'0')

In [69]:
#add colon in between to resemble the format hh:mm
#time_copy2[hrs_row_index] = time_copy2[hrs_row_index].str[:2] + ':' + time_copy2[hrs_row_index].str[2:]
#time_copy2

0    07:20
1    00:09
2    12:30
3    00:04
4    00:01
5    00:14
dtype: object

In [ ]:
time2 = pd.to_datetime(time, format = '%H:%M').dt.time
time2.info()